# Tensorflow: Word2Vec
* This tutorial uses Word2Vec to make predictions of context words based upon a given target word
* Reference: https://www.tensorflow.org/tutorials/text/word2vec

In [7]:
import io, re, string, tqdm, time
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

In [12]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [13]:
SEED = 42
AUTOTUNE = tf.data.AUTOTUNE

## Vectorize an Example Sentence

In [14]:
sentence = "The wide road shimmered in the hot sun"
tokens = list( sentence.lower().split() )
print(len(tokens))
print(tokens)

8
['the', 'wide', 'road', 'shimmered', 'in', 'the', 'hot', 'sun']


In [15]:
vocab = {}
index = 1            # Start indexing from 1
vocab['<pad>'] = 0   # Add a padding token
for token in tokens:
    if token not in vocab:
        vocab[token] = index
        index += 1
        
vocab_size = len(vocab)
print(vocab_size)
print(vocab)

8
{'<pad>': 0, 'the': 1, 'wide': 2, 'road': 3, 'shimmered': 4, 'in': 5, 'hot': 6, 'sun': 7}


In [16]:
inverse_vocab = { index:token for token, index in vocab.items() }
print(inverse_vocab)

{0: '<pad>', 1: 'the', 2: 'wide', 3: 'road', 4: 'shimmered', 5: 'in', 6: 'hot', 7: 'sun'}


In [17]:
example_sequence = [ vocab[w] for w in tokens ]
print(example_sequence)

[1, 2, 3, 4, 5, 1, 6, 7]


## Generate skip-grams from One Sentence

In [19]:
window_size = 2
positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
    example_sequence,
    vocabulary_size=vocab_size,
    window_size=window_size,
    negative_samples=0
    )
print(len(positive_skip_grams))

26


In [20]:
for t,c in positive_skip_grams[:5]:
    print(f"({t},{c}): ({inverse_vocab[t]}, {inverse_vocab[c]})")

(1,7): (the, sun)
(7,6): (sun, hot)
(3,4): (road, shimmered)
(5,1): (in, the)
(6,5): (hot, in)


In [48]:
for t,c in positive_skip_grams:
    if t==1:
        print(f"({t},{c}): ({inverse_vocab[t]}, {inverse_vocab[c]})")

(1,7): (the, sun)
(1,6): (the, hot)
(1,3): (the, road)
(1,2): (the, wide)
(1,4): (the, shimmered)
(1,5): (the, in)


## Negative Sampling for One skip-gram

In [74]:
# Get target and context words for one positive skip-gram
POS_SG_IDX = 0
target_word, context_word = positive_skip_grams[POS_SG_IDX]

# Set the number of negative samples per positive context
num_ns = 4

context_class = tf.reshape( tf.constant( context_word, dtype='int64'), (1,1) )
negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
    true_classes=context_class,  # Class that should be sampled as 'positive'
    num_true=1,                  # Each positive skip-gram has 1 positive context class
    num_sampled=num_ns,          # Number of negative context words to sample
    unique=True,                 # All the negative samples should be unique
    range_max=vocab_size,        # Pick index of the samples from [0, vocab_size)
    seed=SEED,                   # Seed for reproducibility
    name="negative_sampling"     # name of this operation
    )

print( positive_skip_grams[POS_SG_IDX], '=>', [ inverse_vocab[i] for i in  positive_skip_grams[POS_SG_IDX] ] )
print( negative_sampling_candidates )
print( [ inverse_vocab[i.numpy()] for i in negative_sampling_candidates ] )

[1, 7] => ['the', 'sun']
tf.Tensor([5 3 1 6], shape=(4,), dtype=int64)
['in', 'road', 'the', 'hot']


In [75]:
print(tokens)
for t,c in positive_skip_grams:
    if t==1:
        print(f"({t},{c}): ({inverse_vocab[t]}, {inverse_vocab[c]})")

['the', 'wide', 'road', 'shimmered', 'in', 'the', 'hot', 'sun']
(1,7): (the, sun)
(1,6): (the, hot)
(1,3): (the, road)
(1,2): (the, wide)
(1,4): (the, shimmered)
(1,5): (the, in)


## Construct One Training Example

In [76]:
# Add a dimension so you can use concatenation (on the next step)
negative_sampling_candidates = tf.expand_dims( negative_sampling_candidates, 1)

# Concat positive context word with negative sampled words
context = tf.concat( [context_class, negative_sampling_candidates], 0)

# Label first context word as 1 (positive) followed by num_ns 0s (negative)
label = tf.constant([1] + [0]*num_ns, dtype='int64')

# Reshape target to shape (1,) and context and label to (num_ns+1,)
target = tf.squeeze(target_word)
context = tf.squeeze(context)
label = tf.squeeze(label)

print(f"target_index    : {target}")
print(f"target_word     : {inverse_vocab[target_word]}")
print(f"context_indices : {context}")
print(f"context_words   : {[inverse_vocab[c.numpy()] for c in context]}")
print(f"label           : {label}")

target_index    : 1
target_word     : the
context_indices : [7 5 3 1 6]
context_words   : ['sun', 'in', 'road', 'the', 'hot']
label           : [1 0 0 0 0]


In [77]:
print("target  :", target)
print("context :", context)
print("label   :", label)

target  : tf.Tensor(1, shape=(), dtype=int32)
context : tf.Tensor([7 5 3 1 6], shape=(5,), dtype=int64)
label   : tf.Tensor([1 0 0 0 0], shape=(5,), dtype=int64)


## Compile All Steps into One Function

In [78]:
sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(size=10)
print(sampling_table)

[0.00315225 0.00315225 0.00547597 0.00741556 0.00912817 0.01068435
 0.01212381 0.01347162 0.01474487 0.0159558 ]


In [160]:
# Generates skip-gram pairs with negative sampling for a list of sequences
# (int-encoded sentences) based on window size, number of negative samples
# and vocabulary size
def generate_training_data( sequences, window_size, num_ns, vocab_size, seed):
    # Elements of each training example are appended to these lists
    targets, contexts, labels = [], [], []
    
    # Build the samplin table for vocab_size tokens
    sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(vocab_size)
    
    # Iterate over all sequences (sentences) in dataset
    for sequence in tqdm.tqdm(sequences):
        
        # Generate positive skip-gram pairs for a sequence (sentence)
        positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
            sequence,
            vocabulary_size=vocab_size,
            sampling_table=sampling_table,
            window_size=window_size,
            negative_samples=0
            )
        
        # Iterate over each positive skip-gram pair to produce training examples
        # with positive context word and negative samples
        for target_word, context_word in positive_skip_grams:
            context_class = tf.expand_dims( tf.constant([context_word], dtype='int64'), 1)
            negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
                true_classes=context_class,
                num_true=1,
                num_sampled=num_ns,
                unique=True,
                range_max=vocab_size,
                seed=SEED,
                name="negative_sampling"
                )
            
            # Build context and label vectors (for one target word)
            negative_sampling_candidates = tf.expand_dims( negative_sampling_candidates, 1)
            context = tf.concat([ context_class, negative_sampling_candidates], 0)
            label = tf.constant([1] + [0]*num_ns, dtype='int64')
            
            # Append each element from the training example to global lists
            targets.append(target_word)
            contexts.append(context)
            labels.append(label)
            
    return targets, contexts, labels

## Prepare Training Data for Word2Vec

In [161]:
path_to_file = tf.keras.utils.get_file(
                'shakespeare.txt',
                'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [162]:
with open(path_to_file) as f: 
    lines = f.read().splitlines()
for line in lines[:20]:
    print(line)

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.


In [163]:
text_ds = tf.data.TextLineDataset(path_to_file).filter(lambda x: tf.cast(tf.strings.length(x), bool))

In [164]:
print( [ x.numpy().decode() for x in text_ds.take(5) ])

['First Citizen:', 'Before we proceed any further, hear me speak.', 'All:', 'Speak, speak.', 'First Citizen:']


In [165]:
# Now, create a customer standardization function to lowercase the text
# and remove punctuation
def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    no_punct = tf.strings.regex_replace(lowercase, f"[{re.escape(string.punctuation)}]", '')
    return no_punct

# Define the vocabulary size and number of words in a sentence
vocab_size = 4096
sequence_length = 10

# Use the TextVectorization laye to normalize, split, and map strings to integers.
# Set output_sequence_length length to pad all samples to same length.
vectorize_layer = layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length
    )

In [166]:
# Create the vocabulary
vectorize_layer.adapt(text_ds.batch(1024))

In [167]:
# Save the created vocabulary for reference
inverse_vocab = vectorize_layer.get_vocabulary()
print(inverse_vocab[:20])

['', '[UNK]', 'the', 'and', 'to', 'i', 'of', 'you', 'my', 'a', 'that', 'in', 'is', 'not', 'for', 'with', 'me', 'it', 'be', 'your']


In [168]:
# Vectorize the data in text_ds
text_vector_ds = text_ds.batch(1024).prefetch(AUTOTUNE).map(vectorize_layer).unbatch()

In [169]:
print( [ [ v for v in x.numpy() ] for x in text_vector_ds.take(5) ])
print( [ [ inverse_vocab[v] for v in x.numpy() if v != 0 ] for x in text_vector_ds.take(5) ])

[[89, 270, 0, 0, 0, 0, 0, 0, 0, 0], [138, 36, 982, 144, 673, 125, 16, 106, 0, 0], [34, 0, 0, 0, 0, 0, 0, 0, 0, 0], [106, 106, 0, 0, 0, 0, 0, 0, 0, 0], [89, 270, 0, 0, 0, 0, 0, 0, 0, 0]]
[['first', 'citizen'], ['before', 'we', 'proceed', 'any', 'further', 'hear', 'me', 'speak'], ['all'], ['speak', 'speak'], ['first', 'citizen']]


In [170]:
sequences = list( text_vector_ds.as_numpy_iterator() )
print(len(sequences))

32777


In [171]:
for seq in sequences[:5]:
    print(f"{seq} => {[inverse_vocab[i] for i in seq]}")

[ 89 270   0   0   0   0   0   0   0   0] => ['first', 'citizen', '', '', '', '', '', '', '', '']
[138  36 982 144 673 125  16 106   0   0] => ['before', 'we', 'proceed', 'any', 'further', 'hear', 'me', 'speak', '', '']
[34  0  0  0  0  0  0  0  0  0] => ['all', '', '', '', '', '', '', '', '', '']
[106 106   0   0   0   0   0   0   0   0] => ['speak', 'speak', '', '', '', '', '', '', '', '']
[ 89 270   0   0   0   0   0   0   0   0] => ['first', 'citizen', '', '', '', '', '', '', '', '']


In [172]:
[ x.numpy() for x in text_vector_ds.take(2) ]

[array([ 89, 270,   0,   0,   0,   0,   0,   0,   0,   0], dtype=int64),
 array([138,  36, 982, 144, 673, 125,  16, 106,   0,   0], dtype=int64)]

In [173]:
targets, contexts, labels = generate_training_data(
    sequences=sequences,
    window_size=2,
    num_ns=4,
    vocab_size=vocab_size,
    seed=SEED
    )

targets = np.array(targets)
contexts = np.array(contexts)[:,:,0]
labels = np.array(labels)

print('\n')
print(f"targets.shape: {targets.shape}")
print(f"contexts.shape: {contexts.shape}")
print(f"labels.shape: {labels.shape}")

100%|██████████| 32777/32777 [00:15<00:00, 2180.74it/s]




targets.shape: (64952,)
contexts.shape: (64952, 5)
labels.shape: (64952, 5)


In [176]:
print(targets[2])
print(contexts[2])
print(labels[2])

93
[ 89  62 969  50   1]
[1 0 0 0 0]


In [178]:
BATCH_SIZE = 1024
BUFFER_SIZE = 10000
dataset = tf.data.Dataset.from_tensor_slices( ( (targets,contexts), labels ))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
print(dataset)

<BatchDataset shapes: (((1024,), (1024, 5)), (1024, 5)), types: ((tf.int64, tf.int64), tf.int64)>


In [179]:
dataset = dataset.cache().prefetch(buffer_size=AUTOTUNE)
print(dataset)

<PrefetchDataset shapes: (((1024,), (1024, 5)), (1024, 5)), types: ((tf.int64, tf.int64), tf.int64)>


## Model and Training

In [182]:
# Word2Vec Model subclass
class Word2Vec(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim):
        super(Word2Vec, self).__init__()
        self.target_embedding = layers.Embedding(vocab_size, embedding_dim,
                                                 input_length=1, name='w2v_embedding')
        self.context_embedding = layers.Embedding(vocab_size, embedding_dim,
                                                 input_length=num_ns+1)
        
    def call(self, pair):
        # target shape: (batch, dummy?)  -- The dummy axis doesn't exist in TF2.7+
        # context shape: (batch, context)
        target, context = pair
        
        # Remove dummy axis if present to get to
        # target shape: (batch,)
        if len(target.shape) == 2:
            target = tf.squeeze(target, axis=1)
        
        # word_emb shape: (batch, embed)
        word_emb = self.target_embedding(target)
        
        # context_emb shape: (batch, context, embed)
        context_emb = self.context_embedding(context)
        
        # dots shape: (batch, context)
        dots = tf.einsum('be,bce->bc', word_emb, context_emb)
        
        # Return dot production = the prediction of labels
        return dots

In [183]:
# Optional: Custom Loss Function (or can just use tf.keras.losses.CategoricalCrossEntropy)
def custom_loss(x_logit, y_true):
    return tf.nn.sigmoid_cross_entropy_with_logits(logits=x_logit, labels=y_true)

In [185]:
# Instantiate and compile the model
embedding_dim = 128
word2vec = Word2Vec( vocab_size, embedding_dim )
word2vec.compile(optimizer='adam',
                 loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                 metrics=['accuracy']
                )

In [186]:
# Define a callbck to log training statistics for tensorboard
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir='logs')

In [187]:
# Train the model with the dataset prepared above for a number of epochs
word2vec.fit(dataset, epochs=20, callbacks=[tensorboard_callback])

Epoch 1/20
63/63 [==============================] - 3s 26ms/step - loss: 1.6082 - accuracy: 0.2351
Epoch 2/20
63/63 [==============================] - 1s 21ms/step - loss: 1.5888 - accuracy: 0.5567
Epoch 3/20
63/63 [==============================] - 1s 21ms/step - loss: 1.5410 - accuracy: 0.6052
Epoch 4/20
63/63 [==============================] - 1s 22ms/step - loss: 1.4573 - accuracy: 0.5786
Epoch 5/20
63/63 [==============================] - 1s 21ms/step - loss: 1.3576 - accuracy: 0.5832
Epoch 6/20
63/63 [==============================] - 1s 22ms/step - loss: 1.2592 - accuracy: 0.6088
Epoch 7/20
63/63 [==============================] - 1s 22ms/step - loss: 1.1678 - accuracy: 0.6423
Epoch 8/20
63/63 [==============================] - 1s 22ms/step - loss: 1.0833 - accuracy: 0.6754
Epoch 9/20
63/63 [==============================] - 1s 22ms/step - loss: 1.0052 - accuracy: 0.7074
Epoch 10/20
63/63 [==============================] - 1s 22ms/step - loss: 0.9329 - accuracy: 0.7372
Epoch 11/

In [188]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 23932), started 22:14:32 ago. (Use '!kill 23932' to kill it.)

## Embedding Lookup and Analysis

In [189]:
# Obtain model weights and vocabulary
weights = word2vec.get_layer('w2v_embedding').get_weights()[0]
vocab = vectorize_layer.get_vocabulary()

In [191]:
# Create and save the vectors and metadata file
with io.open('./data/vectors.tsv', 'w', encoding='utf-8') as out_v:
    with io.open('./data/metadata.tsv', 'w', encoding='utf-8') as out_m:
        for i,w in enumerate(vocab):
            
            # Skip index 0 => padding
            if i==0:
                continue
                
            vec = weights[i]
            out_v.write( '\t'.join([str(x) for x in vec]) + '\n')
            out_m.write(w + '\n')

## Visualize the Embeddings
* Tensorflow Embedding Projector: http://projector.tensorflow.org/
* Click on "Load data".
* Upload the two files you created above: vecs.tsv and meta.tsv.